# Inteligência Computacional

## Projeto DATA MINING CUP Competition 2013

In [1]:
import pandas as pd

In [2]:
from copy import deepcopy

In [3]:
data = pd.read_csv('transact_train.csv')

## Tratando Granularidade e Criando Variáveis

### Agrupando Dados
agrupando todos os dataframes separados por sessionNo em um vetor

In [4]:
group = data["sessionNo"]
agg = data.groupby([group])
splited = []
for group in agg:
    splited.append(pd.DataFrame(data = group[1], columns=data.columns.values))

In [5]:
# vetor de dataframes separados por sessionNo
#splited[2]

### Criando Variáveis da mudança de granularidade
Essas são as variáveis que criamos buscando recuperar dados perdidos na mudança da granularidade
##### bStep_count
    Quantidade de bStep maiores que 3 de cada sessão
##### countLog
    Numero de log de cada sessão
##### modes
    Moda do bStep de cada Sessão.

In [6]:
bStep_count = []
countLog = []
modes = []
for session in splited:
    countStep = 0
    countLog.append(len(session))
    modes.append(session['bStep'].mode()[0])
    for j in range(len(session)):
        if(session['bStep'].iloc[j] != "?" and int(session['bStep'].iloc[j]) >= 3):
            countStep +=1
    bStep_count.append(countStep)

In [7]:
#bStep_count
#countLog
#modes

### Mudando Granularidade

In [8]:
#Mudando a granularidade
data = data.groupby(['sessionNo'], as_index=False).last()

### Criando Variáveis de Correlação

As técnicas de inteligência artificial pressupõem que as variáveis não têm correlação entre si, apenas relação direta com a variável alvo.

Essas são as variáveis que criamos buscando informar uma correlação entre variáveis
##### diffCounts
    diferença entre cCount e bCount
##### durCount
    Razão entre a duração e o Ccount

In [9]:
# diffCounts
diffCounts = (data['cCount']-data['bCount']).values

# durCount
durCount = (data['duration']/data['cCount'])
durCount = durCount.replace(float("inf"), '?', regex=False).values
# Testar depois com outros valores para tratar o inf

In [10]:
#diffCounts
#durCount

### Atribuindo numeros às classes
Buscando melhorar o processamento, transformamos todos os nomes de classe para numeros, numa sequencia de 0 ao numero de classes -1 para cada classe diferente.

In [11]:
##Ver os valores diferentes de availability
#data.groupby(['availability'], as_index=False).last()

##Ver os valores diferentes de onlineStatus
#data.groupby(['onlineStatus'], as_index=False).last()

##Ver os valores diferentes de order
#data.groupby(['order'], as_index=False).last()

In [12]:
# availability
data.loc[(data.availability == 'completely not determinable'),'availability']= 0
data.loc[(data.availability == 'completely not orderable'),'availability']= 1
data.loc[(data.availability == 'completely orderable'),'availability']= 2
data.loc[(data.availability == 'mainly not determinable'),'availability']= 3
data.loc[(data.availability == 'mainly not orderable'),'availability']= 4
data.loc[(data.availability == 'mainly orderable'),'availability']= 5
data.loc[(data.availability == 'mixed'),'availability']= 6

# onlineStatus
data.loc[(data.onlineStatus == 'y'),'onlineStatus']= 1
data.loc[(data.onlineStatus == 'n'),'onlineStatus']= 0

# order
data.loc[(data.order == 'y'),'order']= 1
data.loc[(data.order == 'n'),'order']= 0

### Inserindo as colunas das variáveis criadas
Com as variáveis criadas, adicionamos ao conjunto de dados com a granularidade já alterada.

In [13]:
#Inserindo colunas de variáveis no conjunto de dados total

#bStep_count
data.insert(23, "bStep_count", bStep_count , True) 

#countLog
data.insert(24, "countLog", countLog , True)

#modes
data.insert(25, "modes", modes , True) 

#diffCounts
data.insert(26, "diffCounts", diffCounts , True) 

#durCount
data.insert(27, "durCount", durCount, True) 

In [14]:
#data

### Tratando Missing Values

In [15]:
backupData = deepcopy(data)

In [16]:
#Tratamento de missing value com o valor minimo

In [17]:
def replaceMissingbyMin(data, field):
    data[field] = data[field].replace('?', data[field].min(), regex=False)


In [18]:
varsNum = ['cMinPrice', 'cMaxPrice', 'cSumPrice', 'bMinPrice', 'bMaxPrice', 'bSumPrice', 'bStep', 'maxVal', 
           'customerScore', 'accountLifetime', 'payments', 'age', 'lastOrder']

for var in varsNum:
    replaceMissingbyMin(data, var)

In [19]:
data2 = deepcopy(data)

In [20]:
def replaceMissingbyMode(data, field):
    mode = data[field].mode()
    if((mode == '?').bool()):
        mode = data[field].value_counts().index[1]
    data[field] = data[field].replace('?', mode, regex=False)

In [21]:
categories = ['onlineStatus', 'availability', 'address']

In [22]:
#substitui missing value da categoria pela moda
for category in categories:
    replaceMissingbyMode(data, category)

In [23]:
#substitui missing value da categoria pela moda
for category in categories:
    replaceMissingbyMode(data, category)

In [24]:
#substitui missing value da categoria por -1
def replaceMissingbyNew(data, field):
    data[field] = data[field].replace('?', -1, regex=False)

In [25]:
for category in categories:
    replaceMissingbyNew(data2, category)

In [26]:
#Salva a tabela
datas = [data, data2]
data = deepcopy(backupData)

In [27]:
data[0]

KeyError: 0

In [ ]:
#Substitui missing value  pelo máximo.